In [1]:
import torch
from torch.autograd import Variable # Pytorch數據格式model
import numpy as np
import torch.nn as nn # nn stand for neural network
import torch.nn.functional as F
from torchvision import datasets,transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# load dataset
# MNIST size is 28*28*1
train_dataset = datasets.MNIST('data/',download=False,train=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,)),
                               ]))
test_dataset = datasets.MNIST('data/',download=False,train=False,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,)),
                              ]))

In [3]:
# load data
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)

In [4]:
# Lenet5 model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16,120,kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120,84)
        self.fc2 = nn.Linear(84,10)
        self.logsoftmax = nn.LogSoftmax()
        
    def forward(self,x):
        in_size = x.size(0)
        out = self.relu(self.mp(self.conv1(x)))
        out = self.relu(self.mp(self.conv2(out)))
        out = self.relu(self.conv3(out))
        out = out.view(in_size, -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return self.logsoftmax(out)

In [5]:
model = CNN_Model()
model.cuda()
print(model)

CNN_Model(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
  (logsoftmax): LogSoftmax(dim=None)
)


In [6]:
loss_fn = nn.NLLLoss() # One way to calculate loss
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [7]:
print(model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[ 0.1103, -0.0662,  0.0349,  0.0106, -0.1341],
          [-0.1156,  0.1037,  0.0193, -0.0072,  0.1909],
          [ 0.1490,  0.1236,  0.0846,  0.1197, -0.0451],
          [-0.0569, -0.1961,  0.0361,  0.1341,  0.1045],
          [-0.1561,  0.1229,  0.1275,  0.1392, -0.0182]]],


        [[[ 0.1454,  0.1587, -0.1107, -0.1326,  0.1149],
          [ 0.0378, -0.0138, -0.1872, -0.0593,  0.1755],
          [ 0.1342, -0.0726,  0.0341, -0.1049,  0.1531],
          [ 0.0636, -0.1464,  0.1557,  0.1257,  0.1425],
          [ 0.1000, -0.0182,  0.1730,  0.1359, -0.0194]]],


        [[[-0.0337, -0.1935, -0.1979, -0.0921, -0.0433],
          [-0.0793, -0.1466, -0.1816, -0.1949,  0.1641],
          [-0.1601,  0.1158, -0.0654,  0.0809,  0.1975],
          [ 0.1938, -0.1134, -0.1088, -0.0981,  0.1839],
          [-0.1901, -0.1329, -0.1824, -0.1150,  0.0830]]],


        [[[ 0.1326, -0.1196, -0.1052, -0.0517,  0.0118],
          [-0.1316, -0.1243,  0.0463,  0.0766,

In [9]:
device = torch.device("cuda")
for epoch in range(20):
    for step, (image, label) in enumerate(train_loader):
        
        image,label = Variable(image),Variable(label)
        image = image.to(device)
        label = label.to(device)
        pred = model(image)
        loss = loss_fn(pred,label)
        print(epoch,step,loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # update parameter

torch.Size([1, 28, 28])


C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 0 2.309048652648926
torch.Size([1, 28, 28])
0 1 2.3063135147094727
torch.Size([1, 28, 28])
0 2 2.3239898681640625
torch.Size([1, 28, 28])
0 3 2.3102028369903564
torch.Size([1, 28, 28])
0 4 2.2916440963745117
torch.Size([1, 28, 28])
0 5 2.305522918701172
torch.Size([1, 28, 28])
0 6 2.314297676086426
torch.Size([1, 28, 28])
0 7 2.3122637271881104
torch.Size([1, 28, 28])
0 8 2.311330795288086
torch.Size([1, 28, 28])
0 9 2.3088650703430176
torch.Size([1, 28, 28])
0 10 2.301898241043091
torch.Size([1, 28, 28])
0 11 2.294861078262329
torch.Size([1, 28, 28])
0 12 2.3053247928619385
torch.Size([1, 28, 28])
0 13 2.3061516284942627
torch.Size([1, 28, 28])
0 14 2.2915291786193848
torch.Size([1, 28, 28])
0 15 2.294494152069092
torch.Size([1, 28, 28])
0 16 2.3078646659851074
torch.Size([1, 28, 28])
0 17 2.3096535205841064
torch.Size([1, 28, 28])
0 18 2.305293321609497
torch.Size([1, 28, 28])
0 19 2.319544553756714
torch.Size([1, 28, 28])
0 20 2.3061392307281494
torch.Size([1, 28, 28])
0 21 2.2961

0 185 2.2703487873077393
torch.Size([1, 28, 28])
0 186 2.2736454010009766
torch.Size([1, 28, 28])
0 187 2.27268648147583
torch.Size([1, 28, 28])
0 188 2.2700858116149902
torch.Size([1, 28, 28])
0 189 2.2544937133789062
torch.Size([1, 28, 28])
0 190 2.25712251663208
torch.Size([1, 28, 28])
0 191 2.2703561782836914
torch.Size([1, 28, 28])
0 192 2.259230375289917
torch.Size([1, 28, 28])
0 193 2.262529134750366
torch.Size([1, 28, 28])
0 194 2.2803268432617188
torch.Size([1, 28, 28])
0 195 2.275827646255493
torch.Size([1, 28, 28])
0 196 2.264312982559204
torch.Size([1, 28, 28])
0 197 2.265042543411255
torch.Size([1, 28, 28])
0 198 2.2606263160705566
torch.Size([1, 28, 28])
0 199 2.2811126708984375
torch.Size([1, 28, 28])
0 200 2.257979154586792
torch.Size([1, 28, 28])
0 201 2.2706005573272705
torch.Size([1, 28, 28])
0 202 2.2668325901031494
torch.Size([1, 28, 28])
0 203 2.263794422149658
torch.Size([1, 28, 28])
0 204 2.2763102054595947
torch.Size([1, 28, 28])
0 205 2.262587070465088
torch.S

0 356 2.145648717880249
torch.Size([1, 28, 28])
0 357 2.146627187728882
torch.Size([1, 28, 28])
0 358 2.1846327781677246
torch.Size([1, 28, 28])
0 359 2.125951051712036
torch.Size([1, 28, 28])
0 360 2.200289249420166
torch.Size([1, 28, 28])
0 361 2.168983221054077
torch.Size([1, 28, 28])
0 362 2.139390707015991
torch.Size([1, 28, 28])
0 363 2.132972240447998
torch.Size([1, 28, 28])
0 364 2.145695209503174
torch.Size([1, 28, 28])
0 365 2.161748170852661
torch.Size([1, 28, 28])
0 366 2.145479679107666
torch.Size([1, 28, 28])
0 367 2.1238858699798584
torch.Size([1, 28, 28])
0 368 2.0970544815063477
torch.Size([1, 28, 28])
0 369 2.170062780380249
torch.Size([1, 28, 28])
0 370 2.1145617961883545
torch.Size([1, 28, 28])
0 371 2.1366724967956543
torch.Size([1, 28, 28])
0 372 2.086050033569336
torch.Size([1, 28, 28])
0 373 2.151853322982788
torch.Size([1, 28, 28])
0 374 2.1108994483947754
torch.Size([1, 28, 28])
0 375 2.139465808868408
torch.Size([1, 28, 28])
0 376 2.152942657470703
torch.Size

...

19 275 0.017047572880983353
torch.Size([1, 28, 28])
19 276 0.023489220067858696
torch.Size([1, 28, 28])
19 277 0.009011092595756054
torch.Size([1, 28, 28])
19 278 0.025749504566192627
torch.Size([1, 28, 28])
19 279 0.0345945842564106
torch.Size([1, 28, 28])
19 280 0.017199575901031494
torch.Size([1, 28, 28])
19 281 0.16317175328731537
torch.Size([1, 28, 28])
19 282 0.007305453531444073
torch.Size([1, 28, 28])
19 283 0.02184087410569191
torch.Size([1, 28, 28])
19 284 0.06002624332904816
torch.Size([1, 28, 28])
19 285 0.03792211413383484
torch.Size([1, 28, 28])
19 286 0.07017156481742859
torch.Size([1, 28, 28])
19 287 0.03656014800071716
torch.Size([1, 28, 28])
19 288 0.04618427902460098
torch.Size([1, 28, 28])
19 289 0.01697160303592682
torch.Size([1, 28, 28])
19 290 0.03657832741737366
torch.Size([1, 28, 28])
19 291 0.014933446422219276
torch.Size([1, 28, 28])
19 292 0.029076088219881058
torch.Size([1, 28, 28])
19 293 0.005069062113761902
torch.Size([1, 28, 28])
19 294 0.08639523386955

19 435 0.12264657765626907
torch.Size([1, 28, 28])
19 436 0.029334664344787598
torch.Size([1, 28, 28])
19 437 0.0148844663053751
torch.Size([1, 28, 28])
19 438 0.004424409940838814
torch.Size([1, 28, 28])
19 439 0.014905713498592377
torch.Size([1, 28, 28])
19 440 0.002298695035278797
torch.Size([1, 28, 28])
19 441 0.016600359231233597
torch.Size([1, 28, 28])
19 442 0.010413403622806072
torch.Size([1, 28, 28])
19 443 0.004004893824458122
torch.Size([1, 28, 28])
19 444 0.014862651005387306
torch.Size([1, 28, 28])
19 445 0.042356282472610474
torch.Size([1, 28, 28])
19 446 0.02078200876712799
torch.Size([1, 28, 28])
19 447 0.018960006535053253
torch.Size([1, 28, 28])
19 448 0.07781936973333359
torch.Size([1, 28, 28])
19 449 0.1463407576084137
torch.Size([1, 28, 28])
19 450 0.021814370527863503
torch.Size([1, 28, 28])
19 451 0.020353825762867928
torch.Size([1, 28, 28])
19 452 0.016685428097844124
torch.Size([1, 28, 28])
19 453 0.012551981955766678
torch.Size([1, 28, 28])
19 454 0.1156491190

torch.Size([1, 28, 28])
19 602 0.018752874806523323
torch.Size([1, 28, 28])
19 603 0.009664873592555523
torch.Size([1, 28, 28])
19 604 0.10807568579912186
torch.Size([1, 28, 28])
19 605 0.04987281560897827
torch.Size([1, 28, 28])
19 606 0.039604902267456055
torch.Size([1, 28, 28])
19 607 0.030461572110652924
torch.Size([1, 28, 28])
19 608 0.061702705919742584
torch.Size([1, 28, 28])
19 609 0.038422487676143646
torch.Size([1, 28, 28])
19 610 0.003948690369725227
torch.Size([1, 28, 28])
19 611 0.02766336500644684
torch.Size([1, 28, 28])
19 612 0.05543002486228943
torch.Size([1, 28, 28])
19 613 0.008202123455703259
torch.Size([1, 28, 28])
19 614 0.01043535303324461
torch.Size([1, 28, 28])
19 615 0.01831008680164814
torch.Size([1, 28, 28])
19 616 0.03191981837153435
torch.Size([1, 28, 28])
19 617 0.05294272303581238
torch.Size([1, 28, 28])
19 618 0.06422799080610275
torch.Size([1, 28, 28])
19 619 0.009842275641858578
torch.Size([1, 28, 28])
19 620 0.03873724117875099
torch.Size([1, 28, 28]

19 773 0.001470401999540627
torch.Size([1, 28, 28])
19 774 0.040762759745121
torch.Size([1, 28, 28])
19 775 0.07851997762918472
torch.Size([1, 28, 28])
19 776 0.015565554611384869
torch.Size([1, 28, 28])
19 777 0.04478594660758972
torch.Size([1, 28, 28])
19 778 0.07931220531463623
torch.Size([1, 28, 28])
19 779 0.1301954686641693
torch.Size([1, 28, 28])
19 780 0.005991451442241669
torch.Size([1, 28, 28])
19 781 0.023633934557437897
torch.Size([1, 28, 28])
19 782 0.008128111250698566
torch.Size([1, 28, 28])
19 783 0.008072818629443645
torch.Size([1, 28, 28])
19 784 0.014616791158914566
torch.Size([1, 28, 28])
19 785 0.012389526702463627
torch.Size([1, 28, 28])
19 786 0.0764203742146492
torch.Size([1, 28, 28])
19 787 0.026377836242318153
torch.Size([1, 28, 28])
19 788 0.023307835683226585
torch.Size([1, 28, 28])
19 789 0.0031256305519491434
torch.Size([1, 28, 28])
19 790 0.015824440866708755
torch.Size([1, 28, 28])
19 791 0.02736845053732395
torch.Size([1, 28, 28])
19 792 0.0261493027210

19 934 0.02815098688006401
torch.Size([1, 28, 28])
19 935 0.01361263357102871
torch.Size([1, 28, 28])
19 936 0.012347215786576271
torch.Size([1, 28, 28])
19 937 0.0208336990326643


In [ ]:
print(model.state_dict())

In [ ]:
# Save model
torch.save(model.state_dict(), "./model/MNIST/Lenet5_model.pt")

In [ ]:
correct = 0
for image, label in test_loader:
    image, label = Variable(image), Variable(label)
    image = image.to(device)
    label = label.to(device)
    output = model(image)
    # get the index of the max log-probability
    # keepdim：讓pred和output維度相同
    #[0]代表最大值,[1]代表最大值的位置
    pred = output.data.max(1, keepdim=True)[1]
    #torch.eq：在相同位置返回True，否則返回False 
    #view_as：使label和pred同個shape
    correct += pred.eq(label.data.view_as(pred)).cpu().sum()
print('{:.3f}%\n'.format(
    100. * correct / len(test_loader.dataset)))